## Convert Patterns to Fake Legal Sentences

In [1]:
import pandas as pd
import re
from collections import defaultdict
from pathlib import Path
import pathlib
import time
import random

In [2]:
# initialising primes and cache them
def isPrime(x):
    count = 0
    for i in range(int(x/2)):
        if x % (i+1) == 0:
            count = count+1
    return count == 1

MIN_PRIME = 0
MAX_PRIME = 1000

CACHED_PRIMES = [i for i in range(MIN_PRIME, MAX_PRIME) if isPrime(i)]

In [3]:
# CHANGE_ME BLOCK
# Change Me_1
INPUT_FILE = r'GROUP_MODEL_TEMPLATE_PATTERN_COUNSEL_CL_for_course.xlsx'
#INPUT_FILE = r'../PATTERNS/CASE_LAW_COUNSEL/GROUP_MODEL_TEMPLATE_PATTERN_COUNSEL_CL_SEP_BATCH_v3.xlsx'


# Change Me_2
MODEL_PATTERN_SHEET = 'ModelPatterns'
NAMED_ENTITIES_SHEET = 'NamedEntities'

VAL_TEST_SKIP_STEP = 3 # take every 3rd sentence into validation and test data

# Change Me_3
CORPUS_DIR_PATH = r'../PATTERNS/CORPUS_GROUPS'
#CORPUS_DIR_PATH = r'../PATTERNS/CORPUS_EXAMPLE'

# Change Me_4
PATTERN_GROUP_LIST = ['G1', 'G2','G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9',
                    'G10', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 
                    'G18', 'G19', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 
                      'G26', 'G27', 'G28']
#PATTERN_GROUP_LIST = ['G1']
#PATTERN_GROUP_LIST = ['G2']


# Change Me_5
#MINI_BATCH_SIZE = 64
MINI_BATCH_SIZE = 16
BATCHES_PER_FILE = 3 # MINI_BATCH_SIZE * BATCHES_PER_FILE sentences for each pattern
# Set RANDOM_STATE to a fix prime value for reproducibale results
RANDOM_STATE = random.choice([i for i in CACHED_PRIMES if MIN_PRIME < i < MAX_PRIME])
print(RANDOM_STATE)

43


In [4]:
#from datetime import datetime
#dt = datetime.now()
#dt.microsecond

In [5]:
TOKENIZATION_STRING = r"[^\s\t,!\"()\*\+,-.\/:;?@\[\]\\`\{\|\}¶§·¸ʺ‶‟”“˗„\']+|[,!\"()\*\+,-.\/:;?@\[\]\\`\{\|\}¶§·¸ʺ‶‟”“˗„\']"
TOKENIZER = re.compile(TOKENIZATION_STRING)
LABEL_TYPE = 'ner'
OTHER = "other" # default unsupported entity_tag

def custom_tokenizer(sentence):
    if type(sentence) == float: return ""
    if len(sentence) == 0: return ""    
    tokens = [match.group() for match in TOKENIZER.finditer(sentence)]
    return tokens

def fix_tokenization(sentence):
    if type(sentence) == float: return ""
    if len(sentence) == 0: return ""
    new_sentence = ' '.join(custom_tokenizer(sentence))
    return new_sentence

sent = "' A. Douglas Brady, Jr., Judge of the Superior Court .'"
new_sent = ' '.join(custom_tokenizer(sent))
print(new_sent)

def get_entity_code_df_dict():
    """Maps each EntityCode to a df that stores dictionary data for this EntityCode."""
    dict_files = NER_DF.DictionaryFileName.unique()
    ec_df_dict = {}
    for dict_file in dict_files:
        file_path = f'./{dict_file}'
        dict_df = pd.read_excel(file_path)
        print(f'{dict_df.shape} : {dict_file}')
        df = NER_DF[NER_DF.DictionaryFileName == dict_file]
        entity_code_list = list(df.EntityCode.unique())
        for code in entity_code_list:
            ec_df_dict [code] = dict_df
            
    return ec_df_dict

def print_list(obj_list):
    for obj in obj_list: 
        print(obj)
        print('~'*30)

' A . Douglas Brady , Jr . , Judge of the Superior Court . '


In [6]:
def get_entity(token):
    """If token is EntityCode then extract entity code (e.g., ATTORNEY) and return True to indicate it is EntityCode.
        Else: return token as is and False as an indicator that it is not EntityCode
    """
    start_pos = token.find('<')
    end_pos = token.find('>')
    if start_pos != -1 and end_pos != -1:
        return token[start_pos+1:end_pos], True
    else: return token, False
    
def is_exception_entity(entity):
    entity_splits = entity.split()
    if len(entity_splits) > 1: return True
    else: return False

def get_entity_base(entity_phrase):
    """Remove numeric suffix from the entity name to get the base.
    Check if entity is an ExceptionEntityName: Law Office of <ATTORNEY_2>.
    ASSUMPTION: that entity_phrase contains exactly ONE <base_entity>.
    Example: LAW_FIRM_5 has the base LAW_FIRM
    """
    
    entity_splits = entity_phrase.split()
    for item in entity_splits:
        entity, is_base = get_entity(item)
        if is_base: break
            
    entity_splits = entity.split('_')
    last = entity_splits[-1]
    base = entity
    if last.isnumeric(): # LAW_FIRM_1, LAW_FIRM_2
        base = "_".join(entity_splits[0:-1])        
        
    return base
        
def get_entity_tag(entity, entity_type, supported=True):
    """Extract entity base (e.g., LAW_FIRM_3 has the base LAW_FIRM).
    Use ner dictionary (NER_DICT) look-up to determine the entity tag for the base.
    Ex: entity_base = LAW_FIRM, entity_tag = lawFirm
    """
    entity_tag = OTHER
    entity_base = get_entity_base(entity)
    
    if entity_type:
        entity_tag = NER_DICT[entity_base]['EntityTag']
        if supported: # account for entity being supported or not for tagging
            is_supported = NER_DICT[entity_base]['isSupported']
            if not is_supported: entity_tag = OTHER

    return entity_tag

def ner_parse_pattern(pattern):
    """Given the pattern, return a list of (token, ner_tag) pairs.
    Example: 
    """
    master_list = []
    token_list = custom_tokenizer(pattern)
    for token in token_list:
        entity, entity_type = get_entity(token)
        entity_tag = get_entity_tag(entity, entity_type, supported=False)
        master_list.append((entity, entity_tag))
    return master_list


def get_exceptions_dict(pdf, pattern):
    sp_dict = {}
    for _, row in pdf.iterrows():
        entity = row.ExceptionEntityName
        entity_tag = row.ExceptionEntityTag
        for pos in range(len(pattern)):
            if pattern.startswith(entity, pos):
                sp_dict[pos] = (entity, entity_tag)
    return sp_dict

def get_exceptions_ner_list(sp_dict, pattern):
    master_list = []
    pos_list = list(sp_dict.keys())
    pos_list.sort(reverse=False)
    
    start = 0
    for pos in pos_list:
        entity = sp_dict[pos][0]
        tag = sp_dict[pos][1]
        if start < pos:
            prec_pattern = pattern[start:pos]
            prec_list = ner_parse_pattern(prec_pattern)
            for item in prec_list:
                master_list.append(item)
                
        master_list.append((entity, tag))
        start = pos + len(entity)
    
    if start < len(pattern):
        tail_pattern = pattern[start:len(pattern)]
        tail_list = ner_parse_pattern(tail_pattern)
        for item in tail_list:
            master_list.append(item)
                
    return master_list
    
def ner_parse_pattern_df(pdf):
    """Given the df with exceptions in ExceptionEntityName for pattern handling, 
    return a list of (token, ner_tag) pairs."""
    
    pattern = pdf.iloc[0].PATTERN
    
    # split pattern into sub-patterns based on the exceptions
    # defined in the ExceptionEntityName column
    sp_dict = get_exceptions_dict(pdf, pattern)
    print('~'*30)
    print(sp_dict)
    master_list = get_exceptions_ner_list(sp_dict, pattern)
    
    return master_list
    

def get_sample_name_list(base, replace = True, how_many=64, random_state=RANDOM_STATE, fixed = False):
    """Sample with replacement; otherwise, may not have enough names to sample.""" 
    if not fixed: random_state = random.choice([i for i in CACHED_PRIMES if MIN_PRIME < i < MAX_PRIME])
    df = ENTITY_CODE_DF_DCT[base]
    ser = df[base]
    ser.dropna(inplace = True) # critical, as the excel files are of uneven row numbers per column
    name_list = list(ser.sample(n=how_many, replace=True, random_state=random_state).values)
    return name_list    

def get_fake_entity_names(token, tag, how_many):
    """Given (token, tag) pair, generate how_many instances of fake entity names.
    Example: (ATTORNEY_1, attorney) will have a list of how_many names to use as a
    fake substitute for ATTORNEY_1
    """
    base = get_entity_base(token)
    entity_name_list = get_sample_name_list(base, replace = True, how_many=how_many, random_state=RANDOM_STATE, fixed = False)
    
    if not is_exception_entity(token): return entity_name_list
    else: # Special Cases such as Law Office of <ATTORNEY_2>: assume only one <entity>
        token_list = []
        
        parts = token.split()
        entity_name = []
        
        prefix_pos = token.find('<')
        suffix_pos = token.find('>') + 1
        
        prefix = token[0:prefix_pos]
        suffix = token[suffix_pos:]
        
        for entity in entity_name_list:
            new_entity = ' '.join(entity.split())
            final_token = prefix.strip() + ' ' + new_entity.strip() + ' ' + suffix.strip()
            token_list.append(final_token.strip())
        return token_list
        
def generate_fake_sentences(ner_token_list, how_many = MINI_BATCH_SIZE):
    unique_entity_list = get_unique_entity_list(ner_token_list)
    fake_entity_dict = defaultdict(list)
    master_fake_list = []
    
    for (token, tag) in unique_entity_list:
        key = (token, tag)
        entity_name_list = get_fake_entity_names(token, tag, how_many=how_many)
        fake_entity_dict[key] = entity_name_list
    
    for idx in range(0, how_many):
        fake_list = []
        for (token, tag) in ner_token_list:
            key = (token, tag)
            if key in fake_entity_dict.keys():
                entity_names = fake_entity_dict[key]
                name = entity_names[idx]
                fake_list.append((name, tag))
            else: fake_list.append((token, tag))
            
        master_fake_list.append(fake_list)
        
    return master_fake_list

def get_unique_entity_list(ner_token_list):
    """Extract unique (ENTITY, EntityTag) pairs in the sentence template"""
    entity_list = []
    result_list = []
    for (token, tag) in ner_token_list:
        if tag != OTHER:
            entity_list.append((token, tag))
    if len(entity_list):
        entity_set = set(entity_list)
        result_list = list(entity_set)
    return result_list

In [7]:
# Test the function
#entity_phrase = 'Office of <ATTORNEY_2> Law Firm'
#base = get_entity_base(entity_phrase)
#print (base)
#entity_name_list = get_fake_entity_names(token=entity_phrase, tag='lawFirm', how_many=1)
#print(entity_name_list)

In [8]:
from collections import defaultdict
def process_non_special_patterns(pg_list = PATTERN_GROUP_LIST, 
                                 merge = False, pattern_src = "cl_counsel", 
                                 conll=True):
    """For a given pattern groupID from the pg_list, generate fake sentences and save them in conll format.
    merge = True : combine output into a single file for different groupIDs
    merge = False: do not combine into a single conll file
    """
    
    pattern_sent_dict = defaultdict(list)
    print(len(pg_list), " : The Number of Pattern Groups to Process.")
    for pattern_group in pg_list:
        print("="*50)
        print(pattern_group, ": Processing this pattern group")
        df = NON_SPECIAL_PATTERN_DF [NON_SPECIAL_PATTERN_DF.GroupID == pattern_group]
        pattern_list = df.PATTERN.unique()
        print(pattern_group, ": The Number of Unique Patterns : ", len(pattern_list))
                      
        file_prefix = pattern_group
        if merge: file_prefix = '_'.join(pg_list)
    
        file_names = [f'{file_prefix}_{pattern_src}_train_iobes.txt', 
                      f'{file_prefix}_{pattern_src}_val_iobes.txt',
                      f'{file_prefix}_{pattern_src}_test_iobes.txt']
        
        # Validation and Test data set do not have to be large for each pattern.
        # Taking only every third sentence might suffice.
        file_skip_dict = {file_names[0]:1, file_names[1]:VAL_TEST_SKIP_STEP, file_names[2]:VAL_TEST_SKIP_STEP} # how many sentences to skip
                      
        no_files = len(file_names)
        how_many = MINI_BATCH_SIZE * no_files * BATCHES_PER_FILE
        print('HOW MANY = ', how_many) # 384 = 64 * 3 * 2

        for pid, pattern in enumerate(pattern_list):
            print('='*30)
            print(pattern_group, ": ", pid, ": PATTERN TEMPLATE: \n\t", pattern)
            print('-'*30)
            ner_token_list = ner_parse_pattern(pattern)
            print(ner_token_list)
            print(pid, '\tNER TOKEN LIST: ', ner_token_list)
            
        
            fake_tokenized_sent_list = generate_fake_sentences(ner_token_list, how_many=how_many)
            #print(pid, '\tFAKE TOKENIZED SENTENCE LIST:', fake_tokenized_sent_list)
            pattern_sent_dict[pattern] = fake_tokenized_sent_list
            
            if conll:
                print(len(fake_tokenized_sent_list), ' fake sentences per file and per pattern')
                start = 0
                sent_per_file = len(fake_tokenized_sent_list) // no_files
                print(pid, "\tPreparing CONLL file format for writing...")
                for file_name in file_names:
                    print(pid, "\t",file_name)
                    end = start + sent_per_file
                    skip = file_skip_dict[file_name]
                    conll_text = convert_fake_sent_to_CONLL_sentence(fake_tokenized_sent_list[start:end:skip])
                    write_to_conll_file(conll_text,  file_name,  dir_path=CORPUS_DIR_PATH, mode="a", verbose = False)
                    start = end
                    
    return pattern_sent_dict
                

def process_special_patterns(pg_list = PATTERN_GROUP_LIST, 
                             merge = False, pattern_src = "cl_counsel", 
                             conll=True):
    """For a given pattern groupID from the pg_list, generate fake sentences and save them in conll format.
    merge = True : combine output into a single file for different groupIDs
    merge = False: do not combine into a single conll file
    """
    print(len(pg_list), " : The Number of Pattern Groups to Process.")
    pattern_sent_dict = defaultdict(list)
    
    for pattern_group in pg_list:
        print("="*50)
        print(pattern_group, ": Processing this pattern group")        
        
        df = SPECIAL_PATTERN_DF [SPECIAL_PATTERN_DF.GroupID == pattern_group]
        pattern_list = df.PATTERN.unique()
        print(pattern_group, ": The Number of Unique Patterns : ", len(pattern_list))
                      
        file_prefix = pattern_group
        if merge: file_prefix = '_'.join(pg_list)
    
        file_names = [f'{file_prefix}_{pattern_src}_train_iobes.txt', 
                      f'{file_prefix}_{pattern_src}_val_iobes.txt',
                      f'{file_prefix}_{pattern_src}_test_iobes.txt']
        
        # Validation and Test data set do not have to be large for each pattern.
        # Taking only every third sentence might suffice.
        file_skip_dict = {file_names[0]:1, file_names[1]:VAL_TEST_SKIP_STEP, file_names[2]:VAL_TEST_SKIP_STEP} # how many sentences to skip        
                      
        no_files = len(file_names)
        how_many = MINI_BATCH_SIZE * no_files * BATCHES_PER_FILE
        print('HOW MANY = ', how_many) # 384 = 64 * 3 * 2

        for pid, pattern in enumerate(pattern_list):
            print('='*30)
            print(pattern_group, ": ", pid, ": PATTERN TEMPLATE: \n\t", pattern)
            print('-'*30)
            
            pdf = df[df.PATTERN == pattern]
            
            ner_token_list = ner_parse_pattern_df(pdf)
            print(pid, '\tNER TOKEN LIST: ', ner_token_list)
        
            fake_tokenized_sent_list = generate_fake_sentences(ner_token_list, how_many=how_many)
            #print(pid, '\tFAKE TOKENIZED SENTENCE LIST:', fake_tokenized_sent_list)
            pattern_sent_dict[pattern] = fake_tokenized_sent_list
            
            if conll:
                print(len(fake_tokenized_sent_list), ' fake sentences per file and per pattern')
                start = 0
                sent_per_file = len(fake_tokenized_sent_list) // no_files
                print(pid, "\tPreparing CONLL file format for writing...")
                for file_name in file_names:
                    print(pid, "\t",file_name)
                    end = start + sent_per_file
                    skip = file_skip_dict[file_name]
                    conll_text = convert_fake_sent_to_CONLL_sentence(fake_tokenized_sent_list[start:end:skip])
                    write_to_conll_file(conll_text,  file_name,  dir_path=CORPUS_DIR_PATH, mode="a", verbose = False)
                    start = end
                
    return pattern_sent_dict

In [15]:
# CONLL-related methods
def process_entity_segment(entity, entity_tag):
    token_list = entity.strip().split()
    if len(token_list) == 1:
        single_text = entity + ' S-' + entity_tag + '\n'
        return single_text
    
    B_tag = ' B-' + entity_tag + '\n'
    I_tag = ' I-' + entity_tag + '\n'
    E_tag = ' E-' + entity_tag + '\n'
    
    B_text = token_list[0] + B_tag
    E_text = token_list[-1] + E_tag
    I_text = ""
    for token in token_list[1:len(token_list)-1]:
        I_text = I_text + token + I_tag
        
    return B_text + I_text + E_text    

def process_other_segment(token):
    text = ""
    
    O_tag = " O" + '\n'
    token_list = token.strip().split()
    for token in token_list:
        text = text + token + O_tag
    return text

def get_conll_tag(entity_tag):
    df = NER_DF[NER_DF.isSupported == True]
    status = entity_tag in list(df.EntityTag)
    if status: return entity_tag
    else: return OTHER
    
def convert_fake_sent_to_CONLL_sentence(fake_tokenized_sent_list):
    text = ""
    
    for ner_token_list in fake_tokenized_sent_list:
        for idx, (token, tag) in enumerate(ner_token_list):
            conll_tag = get_conll_tag(tag)
            if conll_tag == OTHER: 
                conll_token = process_other_segment(str(token).strip())
                text = text + conll_token
            else: text = text + process_entity_segment(token.strip(), conll_tag)
        text = text + '\n'

    return text

def write_to_conll_file(text, file_name="conll_iobes.txt", dir_path=CORPUS_DIR_PATH, mode = 'a', verbose = False):
    path = pathlib.Path(dir_path)
    abs_path = path.absolute()
    if not abs_path.exists():
        abs_path.mkdir()
    file_path = abs_path.joinpath(file_name)
        
    with open(file_path, mode, encoding = "utf-8") as the_file:
        the_file.write(text)
        
    if verbose: print("Saved: ", file_path)
    return

### Step-1: Read and Filter Model Pattern File

In [16]:
MODEL_PATTERN_DF = pd.read_excel(INPUT_FILE, sheet_name = MODEL_PATTERN_SHEET)
NER_DF = pd.read_excel(INPUT_FILE, sheet_name = NAMED_ENTITIES_SHEET)
NER_DF.tail(7)

,Category,EntityCode,EntityTag,isSupported,Example,DictionaryFileName,ToDo
21,Location,STATE_ABBR,state,0,Calif.,US_state_abbrev_names.xlsx,NaN
22,Location,COUNTY,county,0,NaN,US_county_state_names.xlsx,NaN
23,Person,PERSON,person,0,NaN,person_names.xlsx,NaN
24,Person,PERSON_LAST_NAME,person,0,NaN,person_names.xlsx,ADD_COL
25,Employment,GOV_TITLE,employmentTitle,0,NaN,US_employment_titles.xlsx,NaN
26,Organization,PRISON,prison,0,NaN,US_organization.xlsx,NaN
27,Organization,CHURCH,church,0,NaN,US_organization.xlsx,NaN


In [17]:
NER_DF.head(12)

,Category,EntityCode,EntityTag,isSupported,Example,DictionaryFileName,ToDo
0,Litigant,LITIGANT_PERSON,litigant,0,NaN,US_litigants.xlsx,NaN
1,Litigant,LITIGANT_COMPANY,litigant,0,NaN,US_litigants.xlsx,NaN
2,Litigant,LITIGANT_COUNTY,litigant,0,NaN,US_litigants.xlsx,NaN
3,Litigant,LITIGANT_CHURCH,litigant,0,NaN,US_litigants.xlsx,NaN
4,Litigant,LITIGANT_HOSPITAL,litigant,0,NaN,US_litigants.xlsx,NaN
5,Litigant,LITIGANT_INSURANCE_CO,litigant,0,NaN,US_litigants.xlsx,NaN
6,Litigant,LITIGANT_FOUNDATION,litigant,0,NaN,US_litigants.xlsx,NaN
7,Litigant,LITIGANT_ROLE,litigant,0,NaN,US_litigants.xlsx,NaN
8,Litigant,GOV_LITIGANT,litigant,0,NaN,US_litigants.xlsx,NaN
9,Litigant,LITIGANT_LAW_FIRM,litigant,0,NaN,US_litigants.xlsx,NaN


In [18]:
MODEL_PATTERN_DF.tail()

,PID,Reviewed,SME,isSpecial,DataError,ExceptionEntityName,ExceptionEntityTag,isFinal,GroupID,PATTERN,PRE_FILLED_PATTERN,Sentence_TEXT,Sentence_ID
467,M_360,1,0,0,0.0,NaN,NaN,1,G9,"<ATTORNEY_1> , <GOV_TITLE_1> , <ATTORNEY_2> , ...","Xavier Becerra , Attorney General , Renu R . G...","Xavier Becerra , Attorney General , Renu R . G...",S396
468,FP_1,1,0,0,0.0,NaN,NaN,1,G1,"<LITIGANT_PERSON> , in proper person .","Yilian Quintana Ramirez , in proper person .","Yilian Quintana Ramirez , in proper person .",S1
469,M_361,0,0,1,0.0,<PERSON_LAST_NAME> Law Office,lawFirm,1,G3,"<PERSON_LAST_NAME> Law Office and <ATTORNEY> ,...","Zaragoza Law Office and Gina Zaragoza , under ...","Zaragoza Law Office and Gina Zaragoza , under ...",S397
470,FP_7,0,0,1,0.0,"Law Office of <ATTORNEY> , P . L . L . C .",lawFirm,1,G18,"For <LITIGANT_PERSON> , LITIGANT_ROLE> : <ATTO...",NaN,NaN,S11
471,FP_4,0,0,1,0.0,"<ATTORNEY_2> , Attorney at Law",lawFirm,1,G2,"For <LITIGANT_ROLE> : <ATTORNEY> , Law Offices...",NaN,NaN,S7


In [19]:
ENTITY_CODE_DF_DCT = get_entity_code_df_dict()

(61002, 11) : US_litigants.xlsx
(61002, 3) : US_attorney_names.xlsx
(221, 7) : US_law_firms.xlsx
(28889, 2) : US_cities_state_names.xlsx
(55, 5) : US_cities_abbrev_names.xlsx
(55, 3) : US_state_abbrev_names.xlsx
(3241, 2) : US_county_state_names.xlsx
(61002, 8) : person_names.xlsx
(50, 3) : US_employment_titles.xlsx
(6, 2) : US_organization.xlsx


In [20]:
ENTITY_CODE_DF_DCT.keys()

dict_keys(['LITIGANT_PERSON', 'LITIGANT_COMPANY', 'LITIGANT_COUNTY', 'LITIGANT_CHURCH', 'LITIGANT_HOSPITAL', 'LITIGANT_INSURANCE_CO', 'LITIGANT_FOUNDATION', 'LITIGANT_ROLE', 'GOV_LITIGANT', 'LITIGANT_LAW_FIRM', 'PRISON_ID', 'ATTORNEY', 'LAW_FIRM', 'ODD_LAW_FIRM', 'GOVT_ORG_AS_LAW_FIRM', 'AMICUS_CURAE_LAW_FIRM', 'FOUNDATION_LAW_FIRM', 'CITY', 'CITY_ABBR', 'STATE', 'STATE_NAME', 'STATE_ABBR', 'COUNTY', 'PERSON', 'PERSON_LAST_NAME', 'GOV_TITLE', 'PRISON', 'CHURCH'])

In [21]:
# Values are numeric not text
ENTITY_CODE_DF_DCT['PRISON_ID'] = ENTITY_CODE_DF_DCT['PRISON_ID'].astype({"PRISON_ID": int}, errors='ignore')

In [22]:
ENTITY_CODE_DF_DCT['PRISON_ID'].PRISON_ID[0:5]

0    211567.0
1    211688.0
2    211809.0
3    211930.0
4    212051.0
Name: PRISON_ID, dtype: float64

In [23]:
ENTITY_CODE_DF_DCT['LITIGANT_ROLE'].LITIGANT_ROLE[0:10]

0                 APPELLANT
1                 Appellant
2                 appellant
3                APPELLANTS
4                Appellants
5                appellants
6    APPELLANT / PETITIONER
7    Appellant / Petitioner
8    appellant / petitioner
9    APPELLANT - PETITIONER
Name: LITIGANT_ROLE, dtype: object

In [24]:
ENTITY_CODE_DF_DCT['LAW_FIRM'].head()

,LAW_FIRM,UNIQUE_LAW_FIRM,ODD_LAW_FIRM,FOUNDATION_LAW_FIRM,AMICUS_CURAE_LAW_FIRM,GOVT_ORG_AS_LAW_FIRM,LawFirmPattern
0,Traub Lieberman Straus and Shrewsberry LLP,Pacific Merchant Shipping Association,AARP Foundation Litigation,ACLU of Oklahoma Foundation,FREEDOM FOUNDATION,United States Department of Justice,"Law Office of <ATTORNEY> , Attorney at Law"
1,"Joseph , Aleem & Slowik LLC",Office of Disciplinary Counsel,Adams and Reese,FREEDOM FOUNDATION,Freedom from Religion Foundation,"Office of Immigration Litigation , Department ...","The Law Offices of <ATORNEY> , <ATTORNEY>"
2,Lowe and Reilly,Social Security Administration,AKERMAN SENTERFITT,Freedom from Religion Foundation,GENDER JUSTICE,"U . S . Department of Justice , Civil Division...",NaN
3,"Foland , Wickens , Roper , Hofer & Crawford PC",State Public Defender Office,AKERMAN SENTERFITT EIDSON,Hindu American Foundation,Hindu American Foundation,"United States Department of Justice , Office o...",NaN
4,AKERMAN LLP,Securities and Exchange Commission,Akin Gump,Pacific Legal Foundation,Institute for Free Speech,"U . S . Department of Justice , Office of Immi...",NaN


In [25]:
MODEL_PATTERN_DF.head()

,PID,Reviewed,SME,isSpecial,DataError,ExceptionEntityName,ExceptionEntityTag,isFinal,GroupID,PATTERN,PRE_FILLED_PATTERN,Sentence_TEXT,Sentence_ID
0,M_78,0,0,0,0.0,NaN,NaN,1,G13,"<ATTORNEY> , <GOV_TITLE> , <GOVT_ORG_AS_LAW_FI...","Aaron D . Ford , Attorney General , OFFICE OF ...","Aaron D . Ford , Attorney General , OFFICE OF ...",S114
1,M_79,0,0,0,0.0,NaN,NaN,1,G13,"<ATTORNEY> , <GOV_TITLE> , <GOVT_ORG_AS_LAW_FI...","Aaron M . Frey , Attorney General , OFFICE OF ...","Aaron M . Frey , Attorney General , OFFICE OF ...",S115
2,M_80,0,0,0,0.0,NaN,NaN,1,G13,"<ATTORNEY_1> , <ATTORNEY_2> , <ATTORNEY_3> , D...","Amanda Furman , Ernest H . Nomura , Gabriele V...","Amanda Furman , Ernest H . Nomura , Gabriele V...",S116
3,M_81,0,0,0,0.0,NaN,NaN,1,G13,"<ATTORNEY_1> , <GOV_TITLE_1> , argued the caus...","Amy Chung , Deputy Attorney General , argued t...","Amy Chung , Deputy Attorney General , argued t...",S117
4,M_82,0,0,0,0.0,NaN,NaN,1,G13,"<ATTORNEY_1> , <GOV_TITLE_1> , attorney for <L...","Andrew J . Bruck , Acting Attorney General , a...","Andrew J . Bruck , Acting Attorney General , a...",S118


In [26]:
NER_DF.set_index('EntityCode', inplace = True)
NER_DICT = NER_DF.to_dict(orient='index')
print(NER_DICT['ATTORNEY'])
NER_DF.head()

{'Category': 'Attorney', 'EntityTag': 'attorney', 'isSupported': 1, 'Example': nan, 'DictionaryFileName': 'US_attorney_names.xlsx', 'ToDo': nan}


,Category,EntityTag,isSupported,Example,DictionaryFileName,ToDo
EntityCode,,,,,,
LITIGANT_PERSON,Litigant,litigant,0,NaN,US_litigants.xlsx,NaN
LITIGANT_COMPANY,Litigant,litigant,0,NaN,US_litigants.xlsx,NaN
LITIGANT_COUNTY,Litigant,litigant,0,NaN,US_litigants.xlsx,NaN
LITIGANT_CHURCH,Litigant,litigant,0,NaN,US_litigants.xlsx,NaN
LITIGANT_HOSPITAL,Litigant,litigant,0,NaN,US_litigants.xlsx,NaN


In [27]:
print(MODEL_PATTERN_DF.columns)

Index(['PID', 'Reviewed', 'SME', 'isSpecial', 'DataError',
       'ExceptionEntityName', 'ExceptionEntityTag', 'isFinal', 'GroupID',
       'PATTERN', 'PRE_FILLED_PATTERN', 'Sentence_TEXT', 'Sentence_ID'],
      dtype='object')


In [28]:
ROI_COLUMNS = ['isSpecial', 'ExceptionEntityName', 'ExceptionEntityTag', 
               'GroupID','PATTERN']
ERROR_PATTERN_DF = MODEL_PATTERN_DF [MODEL_PATTERN_DF.isFinal == True][ROI_COLUMNS]
print(ERROR_PATTERN_DF.shape)

(262, 5)


In [29]:
#ROI_COLUMNS_2 = ['Reviewed', 'SME', 'DataError', 
#               'isSpecial', 'ExceptionEntityName', 'ExceptionEntityTag', 
#               'GroupID','PATTERN', 'Sentence_TEXT']
#ERROR_PATTERN_DF = MODEL_PATTERN_DF [MODEL_PATTERN_DF.isFinal == True][ROI_COLUMNS_2]
#print(ERROR_PATTERN_DF.shape)

In [30]:
ERROR_PATTERN_DF.drop_duplicates(inplace = True, ignore_index = True)
print(ERROR_PATTERN_DF.shape)

(225, 5)


In [31]:
ERROR_PATTERN_DF.head()

,isSpecial,ExceptionEntityName,ExceptionEntityTag,GroupID,PATTERN
0,0,NaN,NaN,G13,"<ATTORNEY> , <GOV_TITLE> , <GOVT_ORG_AS_LAW_FI..."
1,0,NaN,NaN,G13,"<ATTORNEY_1> , <ATTORNEY_2> , <ATTORNEY_3> , D..."
2,0,NaN,NaN,G13,"<ATTORNEY_1> , <GOV_TITLE_1> , argued the caus..."
3,0,NaN,NaN,G13,"<ATTORNEY_1> , <GOV_TITLE_1> , attorney for <L..."
4,0,NaN,NaN,G13,"<ATTORNEY_1> , <GOV_TITLE_1> , <ATTORNEY_2> , ..."


In [32]:
UNIQUE_PATTERN_LIST = ERROR_PATTERN_DF.PATTERN.unique()
print(len(UNIQUE_PATTERN_LIST))

216


In [33]:
# Process Unambiguous patterns
NON_SPECIAL_PATTERN_DF = ERROR_PATTERN_DF[ERROR_PATTERN_DF.isSpecial == False]
SPECIAL_PATTERN_DF = ERROR_PATTERN_DF[ERROR_PATTERN_DF.isSpecial == True]
NON_SPECIAL_PATTERN_DF.columns

Index(['isSpecial', 'ExceptionEntityName', 'ExceptionEntityTag', 'GroupID',
       'PATTERN'],
      dtype='object')

In [34]:
print(NON_SPECIAL_PATTERN_DF.shape)
print(SPECIAL_PATTERN_DF.shape)

(191, 5)
(34, 5)


In [35]:
# Examine the patterns
#special_pattern_list = SPECIAL_PATTERN_DF.PATTERN.unique()
#non_special_pattern_list = NON_SPECIAL_PATTERN_DF.PATTERN.unique()

#print('Non-Special Error Patterns:')
#print_list(non_special_pattern_list)
#print('-'*50)
#print('Special Error Patterns:')
#print_list(special_pattern_list)

In [36]:

PATTERN_GROUP_LIST = ['G28']
print(PATTERN_GROUP_LIST)

['G28']


In [43]:
# Comment PATTERN_GROUP_LIST here, once debugging is finished
PATTERN_GROUP_LIST = [#'G1', 'G2',
'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9',
                    'G10', 'G11', 'G12', 'G13', 
                    'G14', 'G15', 'G16', 'G17', 
                    'G18', 'G19', 'G20', 'G21', 'G22', 'G23', 'G24', 'G25', 
                      'G26', 'G27', 'G28']
CONLL = True
pattern_sent_dict_non_spec = process_non_special_patterns(pg_list = PATTERN_GROUP_LIST, 
                                                          merge = False, 
                                                          pattern_src = "cl_counsel", conll=CONLL)
print("COMPLETED NON-SPECIAL PATTERNS")

#-------------------------
pattern_sent_dict_special = process_special_patterns(pg_list = PATTERN_GROUP_LIST, 
                                                     merge = False, 
                                                     pattern_src = "cl_counsel", conll=CONLL)
print("COMPLETED SPECIAL PATTERNS")

26  : The Number of Pattern Groups to Process.
G3 : Processing this pattern group
G3 : The Number of Unique Patterns :  0
HOW MANY =  144
G4 : Processing this pattern group
G4 : The Number of Unique Patterns :  4
HOW MANY =  144
G4 :  0 : PATTERN TEMPLATE: 
	 For Commissioner of Social Security , <LITIGANT_ROLE>  : <ATTORNEY> , LEAD ATTORNEY , <GOVT_ORG_AS_LAW_FIRM> , <CITY> , <STATE> .
------------------------------
[('For', 'other'), ('Commissioner', 'other'), ('of', 'other'), ('Social', 'other'), ('Security', 'other'), (',', 'other'), ('LITIGANT_ROLE', 'litigant'), (':', 'other'), ('ATTORNEY', 'attorney'), (',', 'other'), ('LEAD', 'other'), ('ATTORNEY', 'other'), (',', 'other'), ('GOVT_ORG_AS_LAW_FIRM', 'lawFirm'), (',', 'other'), ('CITY', 'city'), (',', 'other'), ('STATE', 'state'), ('.', 'other')]
0 	NER TOKEN LIST:  [('For', 'other'), ('Commissioner', 'other'), ('of', 'other'), ('Social', 'other'), ('Security', 'other'), (',', 'other'), ('LITIGANT_ROLE', 'litigant'), (':', 'other

***
__STOP HERE__

In [39]:
def get_text(tokenized_sent):
    text = ""
    for entity, tag in tokenized_sent:
        text = text + entity.strip() + ' '
    return text
    
pattern_list = []
sent_list = []
for key, val in pattern_sent_dict_special.items():
    #print(key)
    #print('-'*30)
    for tokenized_sent in val[0:1]:
        sent = get_text(tokenized_sent)
        pattern_list.append(key)
        sent_list.append(sent)
        print(sent)
        print('~'*30)
        
for key, val in pattern_sent_dict_non_spec.items():
    #print(key)
    #print('-'*30)
    for tokenized_sent in val[0:1]:
        sent = get_text(tokenized_sent)
        pattern_list.append(key)
        sent_list.append(sent)
        print(sent)
        print('~'*30)

pattern_sent_df = pd.DataFrame(list(zip(sent_list, pattern_list)))
pattern_sent_df.columns = ['Sentence_TEXT', 'PATTERN']
print(pattern_sent_df.shape)
pattern_sent_df.to_excel("../pattern_sent_examples_4.xlsx")

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [38]:
# Save all the Patterns used to Build the model
pattern_file_path = "../PATTERNS/CORPUS_GROUPS/01_GROUP_TEMPLATES_CASE_LAW_1000.xlsx"
ERROR_PATTERN_DF.to_excel(pattern_file_path, sheet_name = "ModelPatterns")
ner_file_path = "../PATTERNS/CORPUS_GROUPS/NamedEntities.xlsx"
NER_DF.to_excel (ner_file_path, sheet_name = "NamedEntities")


#SIZE = 13888
#print('RANDOM_STATE = ', RANDOM_STATE)
print(NON_SPECIAL_PATTERN_DF.shape[0], "\t: Traditional Templates")
print(SPECIAL_PATTERN_DF.shape[0], "\t : Special Templates")
print(len(UNIQUE_PATTERN_LIST), "\t: TOTAL COUNSEL TEMPLATES")
print('-'*30)
#print(SIZE, "\t : Training Data Sentences")
#print(SIZE, "\t : Validation Data Sentences")
#print(SIZE, "\t : Test Data Sentences")
#print(SIZE*3, "\t : TOTAL CORPUS SIZE")

FileNotFoundError: [Errno 2] No such file or directory: '../PATTERNS/CORPUS_GROUPS/01_GROUP_TEMPLATES_CASE_LAW_1000.xlsx'

In [576]:
print('-'*30)
print(f'\nEntity Tags Dictionary Keys: \n{ENTITY_CODE_DF_DCT.keys()}')
print('-'*30)

------------------------------

Entity Tags Dictionary Keys: 
dict_keys(['LITIGANT_PERSON', 'LITIGANT_COMPANY', 'LITIGANT_COUNTY', 'LITIGANT_CHURCH', 'LITIGANT_HOSPITAL', 'LITIGANT_INSURANCE_CO', 'LITIGANT_FOUNDATION', 'LITIGANT_ROLE', 'GOV_LITIGANT', 'LITIGANT_LAW_FIRM', 'ATTORNEY', 'LAW_FIRM', 'ODD_LAW_FIRM', 'GOVT_ORG_AS_LAW_FIRM', 'AMICUS_CURAE_LAW_FIRM', 'FOUNDATION_LAW_FIRM', 'CITY', 'CITY_ABBR', 'STATE', 'STATE_NAME', 'STATE_ABBR', 'COUNTY', 'PERSON', 'PERSON_LAST_NAME', 'GOV_TITLE', 'PRISON', 'CHURCH'])
------------------------------


In [22]:
print('RANDOM_STATE = ', RANDOM_STATE)
#process_non_special_patterns(pg_list = ["G1","G3"], merge = True, pattern_src = "cl_counsel")

RANDOM_STATE =  743


In [559]:
#process_special_patterns(pg_list = ["G1"], merge = False, pattern_src = "cl_counsel")

In [24]:
#pattern_group = "G3"
#df = NON_SPECIAL_PATTERN_DF [NON_SPECIAL_PATTERN_DF.GroupID == pattern_group]
#print(df.shape)
#df.head()

In [44]:
loaded_model_ner = SequenceTagger.load(initial_model)

NameError: name 'SequenceTagger' is not defined